### Francisco Teixeira Rocha Aragão
### 2021031726
### Tp1 - Processamento de Linguagem Natural

#### Objetivo: Testar diferentes variações de parametros para utilização do modelo Word2vec para a tarefa de verificação de analogias utilizando operações de subtração e adição de vetores.

In [1]:
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import ParameterGrid

In [2]:
# abrindo arquivo de corpus
with open('text8') as f:
    data = f.read()

In [3]:
print(data[0:12])

 anarchism o


In [4]:
# organizando entrada, tokenizando em sentenças de tamanho 50 (valor arbitrario)
data_sentences = []
sentences = []
for i in data.split():
    sentences.append(i)

    if len(sentences) == 50:
        data_sentences.append(sentences)
        sentences = []

In [43]:
print(data_sentences[0:11])

[['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst', 'the', 'term', 'is', 'still', 'used', 'in', 'a', 'pejorative', 'way', 'to', 'describe', 'any', 'act', 'that', 'used', 'violent', 'means', 'to', 'destroy', 'the'], ['organization', 'of', 'society', 'it', 'has', 'also', 'been', 'taken', 'up', 'as', 'a', 'positive', 'label', 'by', 'self', 'defined', 'anarchists', 'the', 'word', 'anarchism', 'is', 'derived', 'from', 'the', 'greek', 'without', 'archons', 'ruler', 'chief', 'king', 'anarchism', 'as', 'a', 'political', 'philosophy', 'is', 'the', 'belief', 'that', 'rulers', 'are', 'unnecessary', 'and', 'should', 'be', 'abolished', 'although', 'there', 'are', 'differing'], ['interpretations', 'of', 'what', 'this', 'means', 'anarchism', 'also', 'refers', 'to', 'related'

## EXPLICAR O MOTIVO DESSES PARAMETROS

In [6]:
# preparando hiperparametros para serem usados
hiperparameters = {
    'vector_size': [3, 5, 7],
    'sg': [0, 1], # 1 = skip-gram, 0 = CBOW
    'window': [15, 20, 25],
    'epochs': [5, 8, 10],
}

grid = list(ParameterGrid(hiperparameters))

In [7]:
# preparando informações para teste que estão presentes no arquivo questions-words.txt
test_vectors = []
target_words = []
with open("questions-words.txt") as f:
        for line in f:
            if line.startswith(":"):
                continue
            
            line = line.strip().lower().split(' ')

            if len(line) != 4:
                 continue
            
            test_vectors.append(line[0:3])
            target_words.append(line[3])      

print(len(test_vectors))      

19544


In [8]:
# treinando modelo word2vec com dados de entrada, variando os hiperparametros e calculando erro médio
# a melhor e pior configuração são salvas e impressas ao final
best_config = {}
min_avg_error = 50000

worst_config = {}
max_avg_error = 0

results = []

# dados estão no formato: palavra1 palavra2 palavra3 palavra4
# a ideia é que palavra1 - palavra2 + palavra3 = palavra4
# assim é calculada a similaridade de cosseno entre o resultado de palavra1 - palavra2 + palavra3 com a palavra4
# e assim o erro é calculado como 1 - similaridade
for parameter_configuration in grid:
    total_error = 0.0
    count_test_words_in_vocab = 0

    model = Word2Vec(sentences=data_sentences, vector_size=parameter_configuration['vector_size'], window=parameter_configuration['window'], sg=parameter_configuration['sg'], epochs=parameter_configuration['epochs'])

    for idx in range(len(test_vectors)):

        # Extract the words in the analogy
        word_a, word_b, word_c = test_vectors[idx]
        target = target_words[idx]

        if all(word in model.wv for word in [word_a, word_b, word_c, target]):

            analogy_vector = model.wv[word_a] - model.wv[word_b] + model.wv[word_c]
            
            similarity = cosine_similarity([analogy_vector], [model.wv[target]])[0][0]

            # melhor erro é proximo de 0 
            error = 1 - similarity
            total_error += error
            count_test_words_in_vocab += 1
            """ print(f"Analogy: {word_a} - {word_b} + {word_c} = {target}")
            print(f"Similarity with {target}: {similarity:.4f}, Error: {error:.4f}")
            print() """

    
    average_error = total_error / count_test_words_in_vocab 
    """ print(f"\nAverage Analogy Error: {average_error}") """

    results.append((parameter_configuration, average_error))

    if average_error < min_avg_error:
        best_config = parameter_configuration
        min_avg_error = average_error
        model.save("best_model")
    
    if average_error > max_avg_error:
        worst_config = parameter_configuration
        max_avg_error = average_error


print(f"\nBest Configuration: {best_config}")
print(f"Min Average Error: {min_avg_error}")

print(f"\nWorst Configuration: {worst_config}")
print(f"Max Average Error: {max_avg_error}")




Best Configuration: {'epochs': 10, 'sg': 1, 'vector_size': 3, 'window': 25}
Min Average Error: 0.11449590156785465

Worst Configuration: {'epochs': 5, 'sg': 0, 'vector_size': 5, 'window': 15}
Max Average Error: 0.7583584528408859


Segundo os resultados acima, a melhor configuração de parametros para o modelo Word2vec com o texto de entrada utilizado é a seguinte:
- Dimensão do embedding: 50
- Janela de contexto: 5
- Iterações: 5
- Algoritmo: Skip-gram
--> Resultando em um erro de 0.55 ao avaliar as analogias

In [9]:
# salvando resultados em um arquivo de texto
results.sort(key=lambda x: x[1])

with open("results5.txt", "w") as f:
    for result in results:
        f.write(f"{result[0]} -> {result[1]}\n")

In [10]:
# lendo o arquivo de resultados
with open("results5.txt", "r") as f:
    for line in f:
        print(line)

{'epochs': 10, 'sg': 1, 'vector_size': 3, 'window': 25} -> 0.11449590156785465

{'epochs': 8, 'sg': 1, 'vector_size': 3, 'window': 25} -> 0.12627258985665643

{'epochs': 10, 'sg': 1, 'vector_size': 3, 'window': 20} -> 0.12849591586262585

{'epochs': 8, 'sg': 1, 'vector_size': 3, 'window': 20} -> 0.13624663163420458

{'epochs': 5, 'sg': 1, 'vector_size': 3, 'window': 25} -> 0.14409597154196624

{'epochs': 5, 'sg': 1, 'vector_size': 3, 'window': 20} -> 0.14422278522447513

{'epochs': 10, 'sg': 1, 'vector_size': 3, 'window': 15} -> 0.1447926147206644

{'epochs': 5, 'sg': 1, 'vector_size': 3, 'window': 15} -> 0.1464375747006119

{'epochs': 8, 'sg': 1, 'vector_size': 3, 'window': 15} -> 0.14745602682742742

{'epochs': 10, 'sg': 1, 'vector_size': 5, 'window': 25} -> 0.18509010214225952

{'epochs': 10, 'sg': 1, 'vector_size': 5, 'window': 20} -> 0.19023491457267575

{'epochs': 8, 'sg': 1, 'vector_size': 5, 'window': 25} -> 0.19100256810465152

{'epochs': 8, 'sg': 1, 'vector_size': 5, 'window'

Percebe-se acima que os menores erros foram obtidos fixando o algoritmo, tamanho do embedding e janela de contexto, variando assim apenas a quantidade de iterações. de maneira geral o fator que mais influenciou no erro foram baixas quantidade de iterações aliado a grande dimensões.

Pelos resultados também foi-se visto que o algoritmo Skip-gram foi mais sensível aos parametros, tendo resultados variando de 0.55 até 0.77, enquanto o CBOW variou de 0.77 até 0.89.